In [11]:
import pandas as pd
import numpy as np
from pulp import LpProblem, LpVariable, LpMaximize, lpSum, LpBinary, PULP_CBC_CMD
import os

# 경로 설정
current_dir = os.getcwd()
project_root = os.path.dirname(os.path.dirname(current_dir))
DATA_DIR = os.path.join(project_root, 'data', 'processed')
DATA_DIR_MODELING = os.path.join(project_root, 'data', 'modeling')

# 데이터 로딩
grids = pd.read_csv(os.path.join(DATA_DIR, 'grid_features_with_prediction_all.csv'))
mclp_candidates = pd.read_csv(os.path.join(DATA_DIR_MODELING, 'mclp_selected_optimal.csv'))

# 거리 계산 함수
def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return 6371 * 2 * np.arcsin(np.sqrt(a))

# MCLP 최적화 함수 
def run_mclp_optimization(p):
    candidates = mclp_candidates.copy()
    demand_points = grids[['grid_id', 'center_lat', 'center_lon', 'demand_score']].copy()

    # 변수 생성
    x = LpVariable.dicts('x', candidates.index, cat=LpBinary)
    y = LpVariable.dicts('y', demand_points.index, cat=LpBinary)

    prob = LpProblem('MCLP', LpMaximize)

    R = 1.0  # 커버 반경 (km)

    # 후보지별 커버되는 격자 인덱스 계산
    coverage = {}
    for i, c_row in candidates.iterrows():
        covered_grids = []
        for j, d_row in demand_points.iterrows():
            dist = haversine(c_row['center_lat'], c_row['center_lon'], d_row['center_lat'], d_row['center_lon'])
            if dist <= R:
                covered_grids.append(j)
        coverage[i] = covered_grids

    # 목적함수: 커버 수요 최대화
    prob += lpSum([demand_points.loc[j, 'demand_score'] * y[j] for j in demand_points.index])

    # 제약조건: 설치 개수 p 이하
    prob += lpSum([x[i] for i in candidates.index]) <= p

    # 각 수요지점은 커버된 후보지가 하나라도 있으면 y=1
    for j in demand_points.index:
        prob += y[j] <= lpSum([x[i] for i in candidates.index if j in coverage[i]])

    # 최적화 실행
    solver = PULP_CBC_CMD(msg=False)
    prob.solve(solver)

    # 결과 추출
    selected_indices = [i for i in candidates.index if x[i].varValue == 1]
    selected_sites = candidates.loc[selected_indices]

    covered_demand = sum(demand_points.loc[j, 'demand_score'] * y[j].varValue for j in demand_points.index)

    return selected_sites, covered_demand

# 사용자 입력
p = int(input("설치 개수(p)를 입력하세요(526개 이하): "))

# 최적화 실행
selected_sites, covered_demand = run_mclp_optimization(p)

print(f"\n✅ 추천 입지 {len(selected_sites)}개 (p={p})")
print(selected_sites[['grid_id', 'center_lat', 'center_lon']])
print(f"\n예상 커버 수요: {covered_demand:.1f}")


설치 개수(p)를 입력하세요(526개 이하): 5777

✅ 추천 입지 526개 (p=5777)
          grid_id  center_lat  center_lon
0    GRID_011_036    37.45175    126.9044
1    GRID_011_037    37.45175    126.9100
2    GRID_012_035    37.45625    126.8988
3    GRID_012_036    37.45625    126.9044
4    GRID_014_033    37.46525    126.8876
..            ...         ...         ...
521  GRID_058_066    37.66325    127.0724
522  GRID_059_060    37.66775    127.0388
523  GRID_059_061    37.66775    127.0444
524  GRID_060_061    37.67225    127.0444
525  GRID_061_063    37.67675    127.0556

[526 rows x 3 columns]

예상 커버 수요: 715662.5
